In [9]:
import json
from os import path, getcwd
import pandas as pd

In [10]:
TOP_LEVEL_DIR = getcwd()
TOP_LEVEL_DIR

'/home/warringt'

In [11]:
API_PORT = 8660
API_PORT

8660

In [12]:
MODEL_PARAMS = {
    "args" : [], # Extra command line args for loading the model

    # LLaVA 1.5-13B

    # 8 bits
    #"model_name": "TheBloke/llava-v1.5-13B-GPTQ:gptq-8bit-32g-actorder_True",
    #"model": "TheBloke_llava-v1.5-13B-GPTQ_gptq-8bit-32g-actorder_True",

    # 4 bits
    #"model_name": "TheBloke/llava-v1.5-13B-GPTQ:gptq-4bit-32g-actorder_True",
    #"model": "TheBloke_llava-v1.5-13B-GPTQ_gptq-4bit-32g-actorder_True",

    # 4 bits - 128g - actorder_True
    #"model_name": "TheBloke/llava-v1.5-13B-GPTQ",
    #"model": "TheBloke_llava-v1.5-13B-GPTQ",

    #"args": []
    #"pipeline": "llava-v1.5-13b"


    # MiniGPT-v4 w/ Vicuna v0 13B backend

    "model_name": "j-min/vicuna-13b-v0-merged",
    "model": "j-min_vicuna-13b-v0-merged",
    
    # 8 bits
    #"model_name": "",
    #"model": "",

    # 4 bits
    #"model_name": "anon8231489123/vicuna-13b-GPTQ-4bit-128g",
    #"model": "anon8231489123_vicuna-13b-GPTQ-4bit-128g",
    #"args": ["--wbits", "4", "--groupsize", "128"],
    
    "pipeline": "minigpt4-13b"
}

In [13]:
SERVER_CMD = [
    "bash",
    "start_linux.sh",
    "--model", MODEL_PARAMS['model'],
    "--multimodal-pipeline", MODEL_PARAMS['pipeline'],
    "--disable_exllama",
    "--loader autogptq", "--no_inject_fused_attention", # Fused attention causes an error
    "--api", "--api-port", f"{API_PORT}",
    *MODEL_PARAMS["args"]
]

KeyError: 'args'

In [ ]:
SCRIPT_NAME = "run_vqav2_val_" + MODEL_PARAMS['model']

In [ ]:
SCITAS_PARAMS = f"""#!/bin/bash -l

#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --time 36:00:00
#SBATCH --cpus-per-task=10
#SBATCH --partition=gpu
#SBATCH --qos=gpu
#SBATCH --gres=gpu:2
#SBATCH --mem 64G

cd ~/tgw
{' '.join(SERVER_CMD)} &
cd ~/
ipython {SCRIPT_NAME}.py
"""

In [ ]:
# Generate SCITAS job script
with open(f'{SCRIPT_NAME}.run', 'w+') as job_file:
    job_file.write(SCITAS_PARAMS)

# Generate actual python script
!jupyter nbconvert --to script run_vqav2.ipynb --output {SCRIPT_NAME} \
    -TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags noconvert

[NbConvertApp] Converting notebook run_vqav2.ipynb to script
[NbConvertApp] Writing 4094 bytes to run_vqav2_val_anon8231489123_vicuna-13b-GPTQ-4bit-128g.py


# 1. Loading the questions

In [8]:
%cd {TOP_LEVEL_DIR}/qvlm

#questions_path = 'datasets/VQA_V2/v2_Questions_Test_mscoco/v2_OpenEnded_mscoco_test2015_questions.json'
questions_path = 'datasets/VQA_V2/v2_OpenEnded_mscoco_val2014_questions.json'
questions_json = json.load(open(questions_path))
questions_df = pd.DataFrame(questions_json['questions'])
questions_df

/home/warringt/qvlm


,image_id,question,question_id
0,262148,Where is he looking?,262148000
1,262148,What are the people in the background doing?,262148001
2,262148,What is he on top of?,262148002
3,393225,What website copyrighted the picture?,393225000
4,393225,Is this a creamy soup?,393225001
...,...,...,...
214349,393212,What is the main color in the photo?,393212000
214350,393212,What is the meaning of this sign?,393212001
214351,393212,What is on the sign?,393212002
214352,393212,Does the arrow point left or right?,393212003


# 2. Launching the model server

In [5]:
%cd {TOP_LEVEL_DIR}/tgw

/home/warringt/tgw


In [7]:
import threading
import subprocess
%cd {TOP_LEVEL_DIR}/tgw

def get_model_server_process(params: dict):
  if (not path.exists(params['model'])):
    !python download-model.py {params['model_name']}
  return lambda: subprocess.run(SERVER_CMD, check=True, shell=True, close_fds=True)

/home/warringt/tgw


In [8]:
%cd {TOP_LEVEL_DIR}/qvlm

import socket,time
from eval.connectors import Connector

def wait_for_port(connector: Connector, delay: int = 3, max_retries: int = 1000):
  sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  conn_info = (connector.url, connector.port)
  result = sock.connect_ex(conn_info)
  counter = max_retries
  while (counter >= 0 and result != 0):
    print(f"Port is not open, retrying in {delay}s...\t({max_retries - counter}/{max_retries})")
    time.sleep(delay)
    result = sock.connect_ex(conn_info)
    counter = counter - 1
  
  if (result == 0):
    print("Port is open!")
    sock.close()
  else:
    print(f"Port was not open after n={max_retries} max retries")
    sock.close()
    exit(1)

/home/warringt/qvlm


In [7]:
#%cd {TOP_LEVEL_DIR}/tgw
# Do not uncomment, this doesn't work yet
#threading.Thread(target=get_model_server_process(PARAMS), daemon=True).start()

/home/warringt/tgw


# 3. Computing the responses

In [9]:
%cd {TOP_LEVEL_DIR}/qvlm

/home/warringt/qvlm


In [10]:
#from eval.connectors.llamafile import LlamafileConnector
from eval.connectors.textgenerationwebui import TextGenerationWebUIConnector

connector = TextGenerationWebUIConnector('localhost', API_PORT, prompt_format = '{prompt}\nAnswer with a single word or phrase.')

In [12]:
from eval.evaluation.VQAV2 import VQAV2Evaluator

evaluator = VQAV2Evaluator(questions_df, img_dir='datasets/VQA_V2/val2014')
evaluator.connect(connector)

wait_for_port(connector, delay=3, max_retries = 200)
time.sleep(30) # The server can take some time to keep booting after the port has been opened...
evaluator.get_responses(f'datasets/VQA_V2/val_{MODEL_PARAMS["model"]}_responses.jsonl')

Port is not open, retrying in 3s...	(0/200)


KeyboardInterrupt: 